# Image Classification scenario with RAI Dashboard

The Flower dataset classifies images into 5 distinct flower categories, each commonly found in the United Kingdom.. This example notebook demonstrates how to use a fine-tuned fastai computer vision model on the dataset to evaluate the model in AzureML.

First, we need to specify the version of the RAI components which are available in the workspace. This was specified when the components were uploaded.

In [ ]:
%pip install azure-ai-ml
%pip install fastai

In [ ]:
version_string = "0.0.13"

We also need to give the name of the compute cluster we want to use in AzureML. Later in this notebook, we will create it if it does not already exist:

In [ ]:
compute_name = "ResponsibleAI"

Finally, we need to specify a version for the data and components we will create while running this notebook. This should be unique for the workspace, but the specific value doesn't matter:

In [ ]:
rai_example_version_string = "5"

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

First, we need to upload the datasets to our workspace. We start by creating an `MLClient` for interactions with AzureML:

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace,
)

print(ml_client)

We can now upload the data to AzureML:

# 2. Accessing the Data

We supply the data as a pair of parquet files and accompanying `MLTable` file. We can download them, preprocess them, and take a brief look:

In [ ]:
import os
import pandas as pd
import json

try:
    from urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve

from zipfile import ZipFile

## 2.1 Download Data

In [ ]:
import os
import urllib
from zipfile import ZipFile

# Change to a different location if you prefer
dataset_parent_dir = "./data"

# create data folder if it doesnt exist.
os.makedirs(dataset_parent_dir, exist_ok=True)

# download data
download_url = "https://publictestdatasets.blob.core.windows.net/data/image_test_classification.zip"

# Extract current dataset name from dataset url
dataset_name = os.path.split(download_url)[-1].split(".")[0]
# Get dataset path for later use
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)
os.makedirs(dataset_dir, exist_ok=True)

# Get the data zip file path
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")

# Download the dataset
urllib.request.urlretrieve(download_url, filename=data_file)

# extract files
with ZipFile(data_file, "r") as zip:
    print("extracting files...")
    zip.extractall(path=dataset_dir)
    print("done")


# delete zip file
os.remove(data_file)

## 2.2. Upload the images to Datastore through an AML Data asset (URI Folder) for training an AutomatedML Model

In order to use the data for training in Azure ML, we upload it to our default Azure Blob Storage of our  Azure ML Workspace.

Reference to URI FOLDER data asset example for further details: https://github.com/Azure/azureml-examples/blob/samuel100/data-samples/sdk/assets/data/data.ipynb

In [ ]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

input_test_data_folder = "image-classification-flowers-new"

try:
    uri_folder_data_asset = ml_client.data.get(
        name=input_test_data_folder,
        version=rai_example_version_string,
    )
except Exception:
    my_data = Data(
        path=dataset_dir,
        type=AssetTypes.URI_FOLDER,
        description="Images of Flowers image classification",
        name=input_test_data_folder,
        version=rai_example_version_string,
    )

    uri_folder_data_asset = ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)

## 2.3. Convert the downloaded data to JSONL

In this example, the fridge object dataset is stored in a directory. There are four different folders inside:
- /daisy
- /dandelion
- /rose
- /sunflower
- /tulip

This is the most common data format for multiclass image classification. Each folder title corresponds to the image label for the images contained inside. In order to use this data to create an AzureML MLTable, we first need to convert it to the required JSONL format. Please refer to the [documentation on how to prepare datasets](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-prepare-datasets-for-automl-images).


The following script is creating a .jsonl file in the corresponding MLTable folder. 

In [ ]:
import json
import os

# We'll copy each JSONL file within its related MLTable folder
validation_mltable_path = os.path.join(dataset_parent_dir, "validation-mltable-folder")

# First, let's create the folder if it doesn't exist
os.makedirs(validation_mltable_path, exist_ok=True)

# Path to the combined JSONL file
validation_annotations_file = os.path.join(
    validation_mltable_path, "validation_annotations.jsonl"
)

# Baseline of json line dictionary
json_line_sample = {
    "image_url": uri_folder_data_asset.path,
    "label": "",
}

# Scan each subdirectory and generate a jsonl line per image, combined in a single JSONL file
with open(validation_annotations_file, "w") as combined_f:
    for class_name in os.listdir(dataset_dir):
        sub_dir = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(sub_dir):
            continue

        # Scan each subdirectory
        print(f"Parsing {sub_dir}")
        for image in os.listdir(sub_dir):
            if image.endswith(".jpg"):
                json_line = dict(json_line_sample)
                json_line["image_url"] += f"{class_name}/{image}"
                json_line["label"] = class_name

                # Write each json line to the combined JSONL file
                combined_f.write(json.dumps(json_line) + "\n")

## 2.4. Create MLTable data input for training an AutomatedML Model

Create MLTable data input using the jsonl files created above.

For documentation on creating your own MLTable assets for jobs beyond this notebook, please refer to below resources
- [MLTable YAML Schema](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-mltable) - covers how to write MLTable YAML, which is required for each MLTable asset.
- [Create MLTable data asset](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-data-assets?tabs=Python-SDK#create-a-mltable-data-asset) - covers how to create mltable data asset. 

In [ ]:
def create_ml_table_file(filename):
    return (
        "$schema: https://azureml/sdk-2-0/MLTable.json\n"
        "type: mltable\n"
        "paths:\n"
        " - file: ./{0}\n"
        "transformations:\n"
        "  - read_json_lines:\n"
        "        encoding: utf8\n"
        "        invalid_lines: error\n"
        "        include_path_column: false\n"
        "  - convert_column_types:\n"
        "      - columns: image_url\n"
        "        column_type: stream_info"
    ).format(filename)


def save_ml_table_file(output_path, mltable_file_contents):
    with open(os.path.join(output_path, "MLTable"), "w") as f:
        f.write(mltable_file_contents)


# Create and save validation mltable
validation_mltable_file_contents = create_ml_table_file(
    os.path.basename(validation_annotations_file)
)
save_ml_table_file(validation_mltable_path, validation_mltable_file_contents)

Load some data for a quick view:

In [ ]:
import mltable

tbl = mltable.load(validation_mltable_path)
test_df: pd.DataFrame = tbl.to_pandas_dataframe()

display(test_df)

The label columns contain the classes:

In [ ]:
target_column_name = "label"

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_test_data = "Flower_classification_new_MLTable_1"

try:
    test_data = ml_client.data.get(
        name=input_test_data,
        version=rai_example_version_string,
    )
except Exception:
    test_data = Data(
        path=validation_mltable_path,
        type=AssetTypes.MLTABLE,
        description="RAI Flower classification test data",
        name=input_test_data,
        version=rai_example_version_string,
    )
    ml_client.data.create_or_update(test_data)

# 3. Creating the Model

A restnet50 model trained on approximately 3000 images from the 5 classes is used in this scenario.

To simplify the model creation process, we're going to use a pipeline. We create a directory for the training script:

In [ ]:
import os

os.makedirs("component_src", exist_ok=True)

Next, we write out our script to retrieve the trained model:

In [ ]:
%%writefile component_src/training_script.py

import argparse
import logging
import json
import os
import time
from zipfile import ZipFile

import mlflow
import mlflow.pyfunc

from azureml.core import Run

from fastai.learner import load_learner

from raiutils.common.retries import retry_function

try:
    from urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve

_logger = logging.getLogger(__file__)
logging.basicConfig(level=logging.INFO)

MODEL_NAME = "flower_classification_model.pkl"

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument(
        "--model_output_path", type=str, help="Path to write model info JSON"
    )
    parser.add_argument(
        "--model_base_name", type=str, help="Name of the registered model"
    )
    parser.add_argument(
        "--model_name_suffix", type=int, help="Set negative to use epoch_secs"
    )
    parser.add_argument(
        "--device", type=int, help=(
            "Device for CPU/GPU supports. Setting this to -1 will leverage "
            "CPU, >=0 will run the model on the associated CUDA device id.")
    )

    # parse args
    args = parser.parse_args()

    # return args
    return args


class FetchModel(object):
    def __init__(self):
        pass

    def fetch(self):
        # Blobstore link to the trained model
        url = ("https://publictestdatasets.blob.core.windows.net/models/"+
               "flower_classification_model.zip")
        urlretrieve(url, filename="flower_classification_model.zip")
        data_file = os.path.join("./", "flower_classification_model.zip")

        with ZipFile(data_file, "r") as zip:
            zip.extractall(path="./")
            
        os.remove(data_file)


def main(args):
    current_experiment = Run.get_context().experiment
    tracking_uri = current_experiment.workspace.get_mlflow_tracking_uri()
    _logger.info("tracking_uri: {0}".format(tracking_uri))
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(current_experiment.name)

    _logger.info("Getting device")
    device = args.device

    _logger.info("Loading parquet input")

    # Load the fastai model
    fetcher = FetchModel()
    action_name = "Model download"
    err_msg = "Failed to download model"
    max_retries = 4
    retry_delay = 60
    retry_function(fetcher.fetch, action_name, err_msg,
                   max_retries=max_retries,
                   retry_delay=retry_delay)
    model = load_learner(MODEL_NAME)

    if device >= 0:
        model = model.cuda()

    if args.model_name_suffix < 0:
        suffix = int(time.time())
    else:
        suffix = args.model_name_suffix
    registered_name = "{0}_{1}".format(args.model_base_name, suffix)
    _logger.info(f"Registering model as {registered_name}")

    # Saving model with mlflow
    _logger.info("Saving with mlflow")

    mlflow.fastai.log_model(
        model,
        artifact_path=registered_name,
        registered_model_name=registered_name
    )

    _logger.info("Writing JSON")
    dict = {"id": "{0}:1".format(registered_name)}
    output_path = os.path.join(args.model_output_path, "model_info.json")
    with open(output_path, "w") as of:
        json.dump(dict, fp=of)


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

Now, we can build this into an AzureML component:

In [ ]:
from azure.ai.ml import load_component

yaml_contents = f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: rai_flower_classification_training_component_new_1
display_name: RAI Flower classification training component
version: {rai_example_version_string}
type: command
inputs:
  model_base_name:
    type: string
  model_name_suffix: # Set negative to use epoch_secs
    type: integer
    default: -1
  device: # set to >= 0 to use GPU
    type: integer
    default: 0
outputs:
  model_output_path:
    type: path
code: ./component_src/
environment: azureml://registries/azureml/environments/responsibleai-vision-ubuntu20.04-py38-cpu/versions/44
command: >-
  python training_script.py
  --model_base_name ${{{{inputs.model_base_name}}}}
  --model_name_suffix ${{{{inputs.model_name_suffix}}}}
  --device ${{{{inputs.device}}}}
  --model_output_path ${{{{outputs.model_output_path}}}}
"""

yaml_filename = "TrainingComponent.yaml"

with open(yaml_filename, "w") as f:
    f.write(yaml_contents)

train_component_definition = load_component(source=yaml_filename)

ml_client.components.create_or_update(train_component_definition)

We need a compute target on which to run our jobs. The following checks whether the compute specified above is present; if not, then the compute target is created.

In [ ]:
from azure.ai.ml.entities import AmlCompute

all_compute_names = [x.name for x in ml_client.compute.list()]

if compute_name in all_compute_names:
    print(f"Found existing compute: {compute_name}")
else:
    my_compute = AmlCompute(
        name=compute_name,
        size="STANDARD_DS13_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=3600,
    )
    ml_client.compute.begin_create_or_update(my_compute)
    print("Initiated compute creation")

## 3.1. Running a training pipeline

Now that we have our training component, we can run it. We begin by generating a unique name for the model.

In [ ]:
import time

model_base_name = "flower_classification_model"
model_name_suffix = "12486"
device = -1

Next, we define our training pipeline. This has two components. The first is the training component which we defined above. The second is a component to register the model in AzureML:

In [ ]:
from azure.ai.ml import dsl, Input

train_model_component = ml_client.components.get(
    name="rai_flower_classification_training_component_new_1",
    version=rai_example_version_string,
)


@dsl.pipeline(
    compute=compute_name,
    description="Register Model for RAI Flower classification",
    experiment_name=f"RAI_Flower_classification_Model_Training_{model_name_suffix}",
)
def my_training_pipeline(model_base_name, model_name_suffix, device):
    trained_model = train_component_definition(
        model_base_name=model_base_name,
        model_name_suffix=model_name_suffix,
        device=device,
    )
    trained_model.set_limits(timeout=7200)

    return {}


model_registration_pipeline_job = my_training_pipeline(
    model_base_name, model_name_suffix, device
)

With the training pipeline defined, we can submit it for execution in AzureML. We define a helper function to wait for the job to complete:

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display


def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job

In [ ]:
# This is the actual submission
training_job = submit_and_wait(ml_client, model_registration_pipeline_job)

# 4. Creating the RAI Vision Insights

Now that we have our model, we can generate RAI Vision insights for it. We will need the `id` of the registered model, which will be as follows:

In [ ]:
expected_model_id = f"{model_base_name}_{model_name_suffix}:1"
azureml_model_id = f"azureml:{expected_model_id}"

Next, we load the RAI components, so that we can construct a pipeline:

In [ ]:
test_mltable = Input(
    type="mltable",
    path=f"{input_test_data}:{rai_example_version_string}",
    mode="download",
)

In [ ]:
registry_name = "azureml"
credential = DefaultAzureCredential()

ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name,
)

rai_vision_insights_component = ml_client_registry.components.get(
    name="rai_vision_insights", version=version_string
)

## 4.1 Constructing the pipeline in sdk

We can now specify our pipeline. Complex objects (such as lists of column names) have to be converted to JSON strings before being passed to the components.

In [ ]:
import json
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes


@dsl.pipeline(
    compute=compute_name,
    description="Example RAI computation on Flower classification",
    experiment_name=f"RAI_Flower_classification_RAIInsights_Computation_{model_name_suffix}",
)
def image_classification_pipeline(
    target_column_name,
    test_data,
    classes,
    use_model_dependency,
):
    # Initiate the RAIInsights
    rai_image_job = rai_vision_insights_component(
        task_type="image_classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        test_dataset=test_data,
        target_column_name=target_column_name,
        classes=classes,
        model_type="fastai",
        dataset_type="private",
        use_model_dependency=use_model_dependency,
        # Set this to true to be able to view explanations without computing
        # them on demand - however, running the dashboard may take several hours
        # for several hundred images.
        precompute_explanation=False,
    )
    rai_image_job.set_limits(timeout=720000)

    rai_image_job.outputs.dashboard.mode = "upload"
    rai_image_job.outputs.ux_json.mode = "upload"

    return {
        "dashboard": rai_image_job.outputs.dashboard,
        "ux_json": rai_image_job.outputs.ux_json,
    }

Next, we define the pipeline object itself, and ensure that the outputs will be available for download:

In [ ]:
import uuid
from azure.ai.ml import Output

insights_pipeline_job = image_classification_pipeline(
    target_column_name=target_column_name,
    test_data=test_mltable,
    classes="[]",
    use_model_dependency=True,
)

rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)
insights_pipeline_job.outputs.ux_json = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/ux_json/",
    mode="upload",
    type="uri_folder",
)

And submit the pipeline to AzureML for execution:

In [ ]:
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

The dashboard should appear in the AzureML portal in the registered model view. The following cell computes the expected URI:

In [ ]:
sub_id = ml_client._operation_scope.subscription_id
rg_name = ml_client._operation_scope.resource_group_name
ws_name = ml_client.workspace_name

expected_uri = f"https://ml.azure.com/model/{expected_model_id}/model_analysis?wsid=/subscriptions/{sub_id}/resourcegroups/{rg_name}/workspaces/{ws_name}"

print(f"Please visit {expected_uri} to see your analysis")